<a href="https://colab.research.google.com/github/SombraDiego/-FAHP---Modelo-de-Apoio-a-Decis-o-para-Auditoria-de-C-digo-Fonte/blob/main/fuzzy_ahp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de Apoio a Decisão para Auditoria de Código Fonte




# Com a incorporação da biblioteca Python (pyDecision.algorithm), foi possível desenvolver um modelo de tomada de decisão. Os resultados são derivados da entrada de dados fornecida pelos especialistas. O modelo FAHP desempenhou um papel crucial ao fornecer pesos, facilitando a ordenação dos critérios de risco e módulos do sistema, priorizando-os de acordo. Além disso, o modelo contribuiu significativamente para a análise de consistência.

In [ ]:
!pip install pyDecision
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.7 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0


In [ ]:
# Required Libraries
import numpy as np

# Fuzzy AHP
from pyDecision.algorithm import fuzzy_ahp_method

In [ ]:
# Function: Fuzzy AHP
def fahp(dataset):
    row_sum = []
    s_row   = []
    f_w     = []
    d_w     = []
    #inc_rat = np.array([0, 0, 0, 0.58, 0.9, 1.12, 1.24, 1.32, 1.41, 1.45])
    inc_rat = np.array([0, 0, 0, 0.58, 0.9, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49, 1.51, 1.48, 1.56, 1.57, 1.59]) #Novo índice de consistência randômico
    X       = [(item[0] + 4*item[1] + item[2])/6 for i in range(0, len(dataset)) for item in dataset[i] ]
    X       = np.asarray(X)
    X       = np.reshape(X, (len(dataset), len(dataset)))
    for i in range(0, len(dataset)):
        a, b, c = 1, 1, 1
        for j in range(0, len(dataset[i])):
            d, e, f = dataset[i][j]
            a, b, c = a*d, b*e, c*f
        row_sum.append( (a, b, c) )
    L, M, U = 0, 0, 0
    for i in range(0, len(row_sum)):
        a, b, c = row_sum[i]
        a, b, c = a**(1/len(dataset)), b**(1/len(dataset)), c**(1/len(dataset))
        s_row.append( ( a, b, c ) )
        L = L + a
        M = M + b
        U = U + c
    for i in range(0, len(s_row)):
        a, b, c = s_row[i]
        a, b, c = a*(U**-1), b*(M**-1), c*(L**-1)
        f_w.append( ( a, b, c ) )
        d_w.append( (a + b + c)/3 )
    n_w      = [item/sum(d_w) for item in d_w]
    vector   = np.sum(X*n_w, axis = 1)/n_w
    lamb_max = np.mean(vector)
    cons_ind = (lamb_max - X.shape[1])/(X.shape[1] - 1)
    rc       = cons_ind/inc_rat[X.shape[1]]
    return f_w, d_w, n_w, rc

Critério Pesos:

In [ ]:
# Dataset
dataset = list([
[(1, 1, 1),(6,7,8),(2,3,4),(9,9,9),(4,5,6)],
[(1/8, 1/7, 1/6),(1, 1, 1),(1/6, 1/5, 1/4),(2,3,4),(1/4, 1/3, 1/2)],
[(1/4, 1/3, 1/2),(4,5,6),(1, 1, 1),(6,7,8),(2,3,4)],
[(1/9, 1/9, 1/9),(1/4, 1/3, 1/2),(1/8, 1/7, 1/6),(1, 1, 1),(1/6, 1/5, 1/4)],
[(1/6, 1/5, 1/4),(2,3,4),(1/4, 1/3, 1/2),(4,5,6),(1, 1, 1)],
])

In [ ]:
# Call Fuzzy AHP Function
fuzzy_weights, defuzzified_weights, normalized_weights, rc = fahp(dataset)

In [ ]:
# Fuzzy Weigths (Extensões Sintéticas)
for i in range(0, len(fuzzy_weights)):
  print('r'+str(i+1)+': ', np.around(fuzzy_weights[i], 3))

r1:  [0.371 0.51  0.69 ]
r2:  [0.044 0.064 0.094]
r3:  [0.181 0.264 0.387]
r4:  [0.025 0.033 0.046]
r5:  [0.088 0.13  0.193]


In [ ]:
# Crisp Weigths
for i in range(0, len(defuzzified_weights)):
  print('r'+str(i+1)+': ', round(defuzzified_weights[i], 3))

r1:  0.523
r2:  0.067
r3:  0.277
r4:  0.035
r5:  0.137


In [ ]:
# Normalized Weigths
for i in range(0, len(normalized_weights)):
  print('r'+str(i+1)+': ', round(normalized_weights[i], 3))

r1:  0.503
r2:  0.065
r3:  0.267
r4:  0.033
r5:  0.132


In [ ]:
# Consistency Ratio
print('RC: ' + str(round(rc, 2)))
if (rc > 0.10):
  print('The solution is inconsistent, the pairwise comparisons must be reviewed')
else:
  print('The solution is consistent')

RC: 0.06
The solution is consistent


Vulnerabilidade

In [ ]:
# Dataset
dataset2 = list([
[(1, 1, 1),(1/8, 1/7, 1/6),(1/6, 1/5, 1/4),(1/6, 1/5, 1/4),(1/4, 1/3, 1/2),(1/4, 1/3, 1/2),(1/8, 1/7, 1/6),(1/8, 1/7, 1/6),(1/9, 1/9, 1/9)],
[(6,7,8),(1, 1, 1),(4,5,6),(4,5,6),(4,5,6),(6,7,8),(1, 1, 1),(1, 1, 1),(1/4, 1/3, 1/2)],
[(4,5,6),(1/6, 1/5, 1/4),(1, 1, 1),(1, 1, 1),(2,3,4),(2,3,4),(1/4, 1/3, 1/2),(1/4, 1/3, 1/2),(1/6, 1/5, 1/4)],
[(4,5,6),(1/6, 1/5, 1/4),(1, 1, 1),(1, 1, 1),(2,3,4),(2,3,4),(1/4, 1/3, 1/2),(1/4, 1/3, 1/2),(1/6, 1/5, 1/4)],
[(2,3,4),(1/6, 1/5, 1/4),(1/4, 1/3, 1/2),(1/4, 1/3, 1/2),(1, 1, 1),(1, 1, 1),(1/6, 1/5, 1/4),(1/6, 1/5, 1/4),(1/8, 1/7, 1/6)],
[(2,3,4),(1/8, 1/7, 1/6),(1/4, 1/3, 1/2),(1/4, 1/3, 1/2),(1, 1, 1),(1, 1, 1),(1/6, 1/5, 1/4),(1/6, 1/5, 1/4),(1/9, 1/9, 1/9)],
[(6,7,8),(1, 1, 1),(2,3,4),(2,3,4),(4,5,6),(4,5,6),(1, 1, 1),(1, 1, 1),(1/4, 1/3, 1/2)],
[(6,7,8),(1, 1, 1),(2,3,4),(2,3,4),(4,5,6),(4,5,6),(1, 1, 1),(1, 1, 1),(1/4, 1/3, 1/2)],
[(9,9,9),(2,3,4),(4,5,6),(4,5,6),(6,7,8),(9,9,9),(2,3,4),(2,3,4),(1, 1, 1)]
])

In [ ]:
# Call Fuzzy AHP Function
fuzzy_weights, defuzzified_weights, normalized_weights, rc = fahp(dataset2)

In [ ]:
# Fuzzy Weigths (Extensões Sintéticas)
for i in range(0, len(fuzzy_weights)):
  print('m'+str(i+1)+': ', np.around(fuzzy_weights[i], 3))

m1:  [0.012 0.017 0.025]
m2:  [0.13  0.176 0.243]
m3:  [0.043 0.063 0.095]
m4:  [0.043 0.063 0.095]
m5:  [0.022 0.032 0.047]
m6:  [0.021 0.03  0.043]
m7:  [0.106 0.152 0.215]
m8:  [0.106 0.152 0.215]
m9:  [0.218 0.316 0.442]


In [ ]:
# Crisp Weigths
for i in range(0, len(defuzzified_weights)):
  print('m'+str(i+1)+': ', round(defuzzified_weights[i], 3))

m1:  0.018
m2:  0.183
m3:  0.067
m4:  0.067
m5:  0.034
m6:  0.031
m7:  0.158
m8:  0.158
m9:  0.325


In [ ]:
# Normalized Weigths
for i in range(0, len(normalized_weights)):
  print('m'+str(i+1)+': ', round(normalized_weights[i], 3))

m1:  0.017
m2:  0.176
m3:  0.065
m4:  0.065
m5:  0.032
m6:  0.03
m7:  0.152
m8:  0.152
m9:  0.312


In [ ]:
# Consistency Ratio
print('RC: ' + str(round(rc, 2)))
if (rc > 0.10):
  print('The solution is inconsistent, the pairwise comparisons must be reviewed')
else:
  print('The solution is consistent')

RC: 0.04
The solution is consistent


Code Smells

In [ ]:
# Dataset
dataset2 = list([
[(1, 1, 1),(1/8, 1/7, 1/6),(1/8, 1/7, 1/6),(1/6, 1/5, 1/4),(1, 1, 1),(1/4, 1/3, 1/2),(1, 1, 1),(1/8, 1/7, 1/6),(1/9, 1/9, 1/9)],
[(6,7,8),(1, 1, 1),(1, 1, 1),(4,5,6),(6,7,8),(4,5,6),(9,9,9),(2,3,4),(1, 1, 1)],
[(6,7,8),(1, 1, 1),(1, 1, 1),(2,3,4),(4,5,6),(4,5,6),(6,7,8),(1, 1, 1),(1/4, 1/3, 1/2)],
[(4,5,6),(1/6, 1/5, 1/4),(1/4, 1/3, 1/2),(1, 1, 1),(2,3,4),(2,3,4),(6,7,8),(1/4, 1/3, 1/2),(1/6, 1/5, 1/4)],
[(1, 1, 1),(1/8, 1/7, 1/6),(1/6, 1/5, 1/4),(1/4, 1/3, 1/2),(1, 1, 1),(1, 1, 1),(2,3,4),(1/6, 1/5, 1/4),(1/8, 1/7, 1/6)],
[(2,3,4),(1/6, 1/5, 1/4),(1/6, 1/5, 1/4),(1/4, 1/3, 1/2),(1, 1, 1),(1, 1, 1),(2,3,4),(1/6, 1/5, 1/4),(1/8, 1/7, 1/6)],
[(1, 1, 1),(1/9, 1/9, 1/9),(1/8, 1/7, 1/6),(1/8, 1/7, 1/6),(1/4, 1/3, 1/2),(1/4, 1/3, 1/2),(1, 1, 1),(1/8, 1/7, 1/6),(1/9, 1/9, 1/9)],
[(6,7,8),(1/4, 1/3, 1/2),(1, 1, 1),(2,3,4),(4,5,6),(4,5,6),(6,7,8),(1, 1, 1),(1/4, 1/3, 1/2)],
[(9,9,9),(1, 1, 1),(2,3,4),(4,5,6),(6,7,8),(6,7,8),(9,9,9),(2,3,4),(1, 1, 1)]
])

In [ ]:
# Call Fuzzy AHP Function
fuzzy_weights, defuzzified_weights, normalized_weights, rc = fahp(dataset2)

In [ ]:
# Fuzzy Weigths (Extensões Sintéticas)
for i in range(0, len(fuzzy_weights)):
  print('m'+str(i+1)+': ', np.around(fuzzy_weights[i], 3))

m1:  [0.017 0.022 0.029]
m2:  [0.176 0.231 0.3  ]
m3:  [0.118 0.16  0.218]
m4:  [0.052 0.076 0.112]
m5:  [0.025 0.033 0.045]
m6:  [0.028 0.039 0.055]
m7:  [0.014 0.018 0.025]
m8:  [0.101 0.142 0.202]
m9:  [0.208 0.279 0.366]


In [ ]:
# Crisp Weigths
for i in range(0, len(defuzzified_weights)):
  print('m'+str(i+1)+': ', round(defuzzified_weights[i], 3))

m1:  0.023
m2:  0.236
m3:  0.165
m4:  0.08
m5:  0.034
m6:  0.041
m7:  0.019
m8:  0.148
m9:  0.284


In [ ]:
# Normalized Weigths
for i in range(0, len(normalized_weights)):
  print('m'+str(i+1)+': ', round(normalized_weights[i], 3))

m1:  0.022
m2:  0.229
m3:  0.161
m4:  0.077
m5:  0.033
m6:  0.039
m7:  0.018
m8:  0.144
m9:  0.276


In [ ]:
# Consistency Ratio
print('RC: ' + str(round(rc, 2)))
if (rc > 0.10):
  print('The solution is inconsistent, the pairwise comparisons must be reviewed')
else:
  print('The solution is consistent')

RC: 0.05
The solution is consistent


Bugs:

In [ ]:
# Dataset
dataset3 = list([
[(1, 1, 1),(1/8, 1/7, 1/6),(1/6, 1/5, 1/4),(1/6, 1/5, 1/4),(1/4, 1/3, 1/2),(1/4, 1/3, 1/2),(1, 1, 1),(1/8, 1/7, 1/6),(1/9, 1/9, 1/9)],
[(6,7,8),(1, 1, 1),(2,3,4),(4,5,6),(6,7,8),(6,7,8),(6,7,8),(2,3,4),(1/4, 1/3, 1/2)],
[(4,5,6),(1/4, 1/3, 1/2),(1, 1, 1),(2,3,4),(4,5,6),(4,5,6),(6,7,8),(1, 1, 1),(1/4, 1/3, 1/2)],
[(4,5,6),(1/6, 1/5, 1/4),(1/4, 1/3, 1/2),(1, 1, 1),(2,3,4),(1, 1, 1),(6,7,8),(1/4, 1/3, 1/2),(1/6, 1/5, 1/4)],
[(2,3,4),(1/8, 1/7, 1/6),(1/6, 1/5, 1/4),(1/4, 1/3, 1/2),(1, 1, 1),(1, 1, 1),(2,3,4),(1/6, 1/5, 1/4),(1/8, 1/7, 1/6)],
[(2,3,4),(1/8, 1/7, 1/6),(1/6, 1/5, 1/4),(1, 1, 1),(1, 1, 1),(1, 1, 1),(2,3,4),(1/6, 1/5, 1/4),(1/8, 1/7, 1/6)],
[(1, 1, 1),(1/8, 1/7, 1/6),(1/8, 1/7, 1/6),(1/8, 1/7, 1/6),(1/4, 1/3, 1/2),(1/4, 1/3, 1/2),(1, 1, 1),(1/8, 1/7, 1/6),(1/9, 1/9, 1/9)],
[(6,7,8),(1/4, 1/3, 1/2),(1, 1, 1),(2,3,4),(4,5,6),(4,5,6),(6,7,8),(1, 1, 1),(1/4, 1/3, 1/2)],
[(9,9,9),(2,3,4),(2,3,4),(4,5,6),(6,7,8),(6,7,8),(9,9,9),(2,3,4),(1, 1, 1)]
])

In [ ]:
# Call Fuzzy AHP Function
fuzzy_weights, defuzzified_weights, normalized_weights, rc = fahp(dataset3)

In [ ]:
# Fuzzy Weigths (Extensões Sintéticas)
for i in range(0, len(fuzzy_weights)):
  print('m'+str(i+1)+': ', np.around(fuzzy_weights[i], 3))

m1:  [0.015 0.02  0.029]
m2:  [0.156 0.231 0.338]
m3:  [0.093 0.135 0.2  ]
m4:  [0.046 0.066 0.099]
m5:  [0.025 0.037 0.054]
m6:  [0.03  0.042 0.059]
m7:  [0.014 0.019 0.026]
m8:  [0.097 0.14  0.207]
m9:  [0.215 0.311 0.438]


In [ ]:
# Crisp Weigths
for i in range(0, len(defuzzified_weights)):
  print('m'+str(i+1)+': ', round(defuzzified_weights[i], 3))

m1:  0.021
m2:  0.242
m3:  0.143
m4:  0.07
m5:  0.039
m6:  0.043
m7:  0.02
m8:  0.148
m9:  0.321


In [ ]:
# Normalized Weigths
for i in range(0, len(normalized_weights)):
  print('m'+str(i+1)+': ', round(normalized_weights[i], 3))

m1:  0.02
m2:  0.231
m3:  0.136
m4:  0.067
m5:  0.037
m6:  0.041
m7:  0.019
m8:  0.141
m9:  0.307


In [ ]:
# Consistency Ratio
print('RC: ' + str(round(rc, 2)))
if (rc > 0.10):
  print('The solution is inconsistent, the pairwise comparisons must be reviewed')
else:
  print('The solution is consistent')

RC: 0.06
The solution is consistent


Plagio/similaridade:

In [ ]:
# Dataset
dataset4 = list([
[(1, 1, 1),(1/8, 1/7, 1/6),(1, 1, 1),(1, 1, 1),(4,5,6),(4,5,6),(1, 1, 1),(2,3,4),(1, 1, 1)],
[(6,7,8),(1, 1, 1),(1, 1, 1),(1, 1, 1),(4,5,6),(4,5,6),(1, 1, 1),(2,3,4),(1, 1, 1)],
[(1, 1, 1),(1, 1, 1),(1, 1, 1),(1, 1, 1),(4,5,6),(4,5,6),(1, 1, 1),(1, 1, 1),(1/4, 1/3, 1/2)],
[(1, 1, 1),(1, 1, 1),(1, 1, 1),(1, 1, 1),(2,3,4),(2,3,4),(1, 1, 1),(1, 1, 1),(1/4, 1/3, 1/2)],
[(1/6, 1/5, 1/4),(1/6, 1/5, 1/4),(1/6, 1/5, 1/4),(1/4, 1/3, 1/2),(1, 1, 1),(1/4, 1/3, 1/2),(1/4, 1/3, 1/2),(1/4, 1/3, 1/2),(1/4, 1/3, 1/2)],
[(1/6, 1/5, 1/4),(1/6, 1/5, 1/4),(1/6, 1/5, 1/4),(1/4, 1/3, 1/2),(2,3,4),(1, 1, 1),(1/4, 1/3, 1/2),(1/4, 1/3, 1/2),(1/6, 1/5, 1/4)],
[(1, 1, 1),(1, 1, 1),(1, 1, 1),(1, 1, 1),(2,3,4),(2,3,4),(1, 1, 1),(1, 1, 1),(1, 1, 1)],
[(1/4, 1/3, 1/2),(1/4, 1/3, 1/2),(1, 1, 1),(1, 1, 1),(2,3,4),(2,3,4),(1, 1, 1),(1, 1, 1),(1/4, 1/3, 1/2)],
[(1, 1, 1),(1, 1, 1),(2,3,4),(2,3,4),(2,3,4),(4,5,6),(1, 1, 1),(2,3,4),(1, 1, 1)]
])

In [ ]:
# Call Fuzzy AHP Function
fuzzy_weights, defuzzified_weights, normalized_weights, rc = fahp(dataset4)

In [ ]:
# Fuzzy Weigths (Extensões Sintéticas)
for i in range(0, len(fuzzy_weights)):
  print('m'+str(i+1)+': ', np.around(fuzzy_weights[i], 3))

m1:  [0.098 0.124 0.155]
m2:  [0.151 0.191 0.238]
m3:  [0.098 0.12  0.15 ]
m4:  [0.084 0.107 0.137]
m5:  [0.021 0.03  0.047]
m6:  [0.026 0.036 0.054]
m7:  [0.098 0.121 0.148]
m8:  [0.062 0.084 0.118]
m9:  [0.134 0.185 0.246]


In [ ]:
# Crisp Weigths
for i in range(0, len(defuzzified_weights)):
  print('m'+str(i+1)+': ', round(defuzzified_weights[i], 3))

m1:  0.126
m2:  0.193
m3:  0.123
m4:  0.11
m5:  0.033
m6:  0.039
m7:  0.123
m8:  0.088
m9:  0.188


In [ ]:
# Normalized Weigths
for i in range(0, len(normalized_weights)):
  print('m'+str(i+1)+': ', round(normalized_weights[i], 3))

m1:  0.123
m2:  0.189
m3:  0.12
m4:  0.107
m5:  0.032
m6:  0.038
m7:  0.12
m8:  0.086
m9:  0.184


In [ ]:
# Consistency Ratio
print('RC: ' + str(round(rc, 2)))
if (rc > 0.10):
  print('The solution is inconsistent, the pairwise comparisons must be reviewed')
else:
  print('The solution is consistent')

RC: 0.08
The solution is consistent


Licença de software indevida:

In [ ]:
# Dataset
dataset5 = list([
[(1, 1, 1),(4,5,6),(4,5,6),(2,3,4),(6,7,8),(6,7,8),(4,5,6),(6,7,8),(1/4, 1/3, 1/2)],
[(1/6, 1/5, 1/4),(1, 1, 1),(1, 1, 1),(1/4, 1/3, 1/2),(4,5,6),(4,5,6),(2,3,4),(4,5,6),(1/8, 1/7, 1/6)],
[(1/6, 1/5, 1/4),(1, 1, 1),(1, 1, 1),(1/6, 1/5, 1/4),(4,5,6),(4,5,6),(1, 1, 1),(4,5,6),(1/8, 1/7, 1/6)],
[(1/4, 1/3, 1/2),(2,3,4),(4,5,6),(1, 1, 1),(6,7,8),(6,7,8),(4,5,6),(6,7,8),(1/6, 1/5, 1/4)],
[(1/8, 1/7, 1/6),(1/6, 1/5, 1/4),(1/6, 1/5, 1/4),(1/8, 1/7, 1/6),(1, 1, 1),(1, 1, 1),(1/4, 1/3, 1/2),(1, 1, 1),(1/9, 1/9, 1/9)],
[(1/8, 1/7, 1/6),(1/6, 1/5, 1/4),(1/6, 1/5, 1/4),(1/8, 1/7, 1/6),(1, 1, 1),(1, 1, 1),(1/4, 1/3, 1/2),(1, 1, 1),(1/9, 1/9, 1/9)],
[(1/6, 1/5, 1/4),(1/4, 1/3, 1/2),(1, 1, 1),(1/6, 1/5, 1/4),(2,3,4),(2,3,4),(1, 1, 1),(2,3,4),(1/8, 1/7, 1/6)],
[(1/8, 1/7, 1/6),(1/6, 1/5, 1/4),(1/6, 1/5, 1/4),(1/8, 1/7, 1/6),(1, 1, 1),(1, 1, 1),(1/4, 1/3, 1/2),(1, 1, 1),(1/9, 1/9, 1/9)],
[(2,3,4),(6,7,8),(6,7,8),(4,5,6),(9,9,9),(9,9,9),(6,7,8),(9,9,9),(1, 1, 1)]
])

In [ ]:
# Call Fuzzy AHP Function
fuzzy_weights, defuzzified_weights, normalized_weights, rc = fahp(dataset5)

In [ ]:
# Fuzzy Weigths (Extensões Sintéticas)
for i in range(0, len(fuzzy_weights)):
  print('m'+str(i+1)+': ', np.around(fuzzy_weights[i], 3))

m1:  [0.157 0.222 0.313]
m2:  [0.056 0.078 0.11 ]
m3:  [0.05  0.065 0.087]
m4:  [0.11  0.155 0.22 ]
m5:  [0.017 0.021 0.029]
m6:  [0.017 0.021 0.029]
m7:  [0.034 0.049 0.071]
m8:  [0.017 0.021 0.029]
m9:  [0.28  0.365 0.472]


In [ ]:
# Crisp Weigths
for i in range(0, len(defuzzified_weights)):
  print('m'+str(i+1)+': ', round(defuzzified_weights[i], 3))

m1:  0.231
m2:  0.081
m3:  0.067
m4:  0.162
m5:  0.022
m6:  0.022
m7:  0.051
m8:  0.022
m9:  0.372


In [ ]:
# Normalized Weigths
for i in range(0, len(normalized_weights)):
  print('m'+str(i+1)+': ', round(normalized_weights[i], 3))

m1:  0.224
m2:  0.079
m3:  0.065
m4:  0.157
m5:  0.022
m6:  0.022
m7:  0.05
m8:  0.022
m9:  0.361


In [ ]:
# Consistency Ratio
print('RC: ' + str(round(rc, 2)))
if (rc > 0.10):
  print('The solution is inconsistent, the pairwise comparisons must be reviewed')
else:
  print('The solution is consistent')

RC: 0.07
The solution is consistent
